In [1]:
from netgen.occ import *
from ngsolve import Mesh
from ngsolve.webgui import Draw

import sys
sys.path.append("../build/rigid_body")
from rigid_body import *

import pythreejs as p3

import numpy as np
from IPython.display import display
import time

In [2]:
# set up OCC CAD model
box = Box(Pnt(0,0,0), Pnt(1,1,1))
geo = OCCGeometry(box)
mesh = Mesh(geo.GenerateMesh(maxh=0.5))

In [3]:
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [4]:
def extract_vertices(mesh: Mesh):
    "extracts a p3js compatible vertex list from a netgen.occ Mesh"
    combined_vertices = [] # flat array of vertices of all faces
    all_vertices = [vert.point for vert in mesh.vertices] # all available vertices

    # throw all vertices of all faces into combined_vertices
    for face in mesh.faces:
        for vertex in face.vertices:
            point = all_vertices[vertex.nr]
            combined_vertices.append(point)
            
    return np.array(combined_vertices, dtype=np.float32)

In [5]:
view_width = 600
view_height = 400

# set up pythreejs 3d object
original_vertices = extract_vertices(mesh)
buffergeom = p3.BufferGeometry(attributes = {"position" : p3.BufferAttribute(original_vertices, normalized=False)})
material = p3.MeshPhongMaterial(color='#ff3333', shininess=150, morphTargets=True, side="DoubleSide")
p3mesh = p3.Mesh(buffergeom, material, position=(0,0,0))

# extra scene contents
camera = p3.PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = p3.DirectionalLight(position=[0, 10, 10])
ambient_light = p3.AmbientLight()

# set up scene
scene = p3.Scene(children=[p3mesh, camera, key_light, ambient_light])
controller = p3.OrbitControls(controlling=camera)
renderer = p3.Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height, antialias=True) # if performance is bad, try removing antalias=True

In [ ]:
r = RigidBody()
r.setMass(mass_cube())
t = Transformation()
t.setTranslation(0.01,0,0)
t.setRotation(0,2,0.005)
t.setRotation(0,1,0.005)
r.dq=t

display(renderer)

current_vertices = original_vertices

# set up simulation object
# rbody = rigid_body.RigidBody()
p3mesh.geometry = p3.BufferGeometry(attributes = {"position" : p3.BufferAttribute(current_vertices, normalized=False)})
p3mesh.matrixAutoUpdate = False
p3mesh.matrixWorldAutoUpdate = False
for i in range(10000):
    t = i/30
    r.simulate(10.0,100)
    # update vertex positions
    # rigid_body.update(original_vertices.data, t)
    #p3mesh.rotateX(0.2)
    #p3mesh.matrix=(1.0, 0.0, 0.0, 0.0, 0.0, -0.6153524829547223, 0.7882520673753188, 0.0, 0.0, -0.7882520673753188, -0.6153524829547223, 0.0, 0,t, 0.0, 1.0)
    p3mesh.matrix=r.q.asTuple()
    print(p3mesh.matrix)
    #time.sleep(1/0)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

(0.9993750651564222, -0.02499479090063223, -0.024994790885916177, 0.0, 0.024994790900599976, 0.9996875325780274, -0.0003124674230792057, 0.0, 0.024994790885948432, -0.0003124674204986564, 0.9996875325783945, 0.0, 0.12531767652851444, 0.012809862871551883, 0.0128098628653008, 1.0)
(0.997501041712754, -0.04995834157150735, -0.049958341539681164, 0.0, 0.049958341571379426, 0.9987505208555842, -0.0012494791461804442, 0.0, 0.04995834153980909, -0.0012494791410656704, 0.9987505208571698, 0.0, 0.2512911375880287, 0.02622864992849476, 0.026228649914346303, 1.0)
(0.9943802719540593, -0.07485945082587699, -0.0748594507769022, 0.0, 0.07485945082559503, 0.9971901359752019, -0.002809864026726013, 0.0, 0.07485945077718413, -0.0028098640192148216, 0.9971901359788573, 0.0, 0.3779504132215805, 0.04023958943494541, 0.04023958941238585, 1.0)
(0.9900166564357982, -0.0996669955222379, -0.0996669954561336, 0.0, 0.09966699552174887, 0.9950083282146128, -0.004991671786982977, 0.0, 0.09966699545662264, -0.0049

In [ ]:
r = RigidBody()
r.setMass(mass_cube())
t = Transformation()
t.setTranslation(0.1,0,0)
#t.setRotation(2,1,0.001)
r.dq=t

display(renderer)

current_vertices = original_vertices

# set up simulation object
# rbody = rigid_body.RigidBody()
p3mesh.geometry = p3.BufferGeometry(attributes = {"position" : p3.BufferAttribute(current_vertices, normalized=False)})
p3mesh.matrixAutoUpdate = False
p3mesh.matrixWorldAutoUpdate = False


In [ ]:
print(r.q)
r.simulate(100,100)

In [ ]:
dir(p3mesh)

In [ ]:
print(p3mesh.matrix)
print(r.q.asTuple())

In [ ]:
for i in range(10000):
    t = i/30
    r.simulate(100,100)
    # update vertex positions
    # rigid_body.update(original_vertices.data, t)

    p3mesh.matrix=r.q.asTuple()
    print(r.q.asTuple())
    time.sleep(1/30)